In [19]:
## Libraries
import pandas as pd
import numpy as np

### 2. Write a function ‘exists’ which takes a variable symbol v and returns whether v is defined.

In [2]:
try:
    var
except NameError:
    print('v is not defined')
else:
    print("exists")

### 3. Create a function to compute N layer of a Pascal Triangle. 

In [26]:
def pascal_triangle(n):
    pascal = [[1]*(i+1) for i in range(n)] # set the initial layer
    
    # O(n^2)
    for i in range(n):
        for j in range(1,i):
            pascal[i][j] = pascal[i-1][j-1] + pascal[i-1][j]
    return pascal

In [50]:
# test cases for pascal triangle
pascal_triangle(10)
# pascal_triangle(20)

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1]]

### 4. Assume have the following portfolio as of 2016/01/01:
    AAPL.O 15%
    IBM.N 20%
    GOOG.O 20%
    BP.N 15%
    XOM.N 10%
    COST.O 15%
    GS.N 5%

a. Using historical daily returns (Yahoo/Google Finance or any other market data
source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

b. Using expected mean, covariance matrix and parametric method, calculate VaR95%
and CVaR95%

c. Assume you can change weights, allow shorting but no leverage (i.e. sum of weights
equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of
each month till end of 2016
Notes: If you have other assumption(s) please state clearly

In [47]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['date'] = pd.to_datetime(portfolio['date'], format='%d/%m/%Y')
portfolio.iloc[:, 1:8] = portfolio.iloc[:, 1:8].pct_change(1) # daily returns

In [48]:
portfolio

,date,aapl,ibm,goog,bp,xom,cost,gs
0,2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-05,-0.025057,-0.000736,0.000998,-0.004185,0.008521,0.002445,-0.017218
2,2016-01-06,-0.019470,-0.005006,0.001401,-0.020369,-0.008321,-0.009254,-0.024413
3,2016-01-07,-0.042494,-0.017090,-0.023170,-0.028713,-0.016006,-0.022909,-0.030735
4,2016-01-08,0.005392,-0.009258,-0.016410,-0.017669,-0.020202,-0.017504,-0.004131
...,...,...,...,...,...,...,...,...
247,2016-12-23,0.002064,-0.002095,-0.001706,0.002980,-0.001761,0.000062,0.003540
248,2016-12-27,0.006522,0.002579,0.002076,0.002431,0.000441,-0.000185,0.002448
249,2016-12-28,-0.004434,-0.005684,-0.008212,0.003773,-0.004959,-0.006418,-0.003767
250,2016-12-29,-0.000343,0.002467,-0.002879,0.004027,0.000554,0.000745,-0.010264
